In [1]:
!pip install -r requirements.txt

   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
    --------------------------------------- 0.3/15.8 MB ? eta -:--:--
    --------------------------------------- 0.3/15.8 MB ? eta -:--:--
   - -------------------------------------- 0.5/15.8 MB 598.5 kB/s eta 0:00:26
   - -------------------------------------- 0.5/15.8 MB 598.5 kB/s eta 0:00:26
   - -------------------------------------- 0.8/15.8 MB 609.6 kB/s eta 0:00:25
   - -------------------------------------- 0.8/15.8 MB 609.6 kB/s eta 0:00:25
   -- ------------------------------------- 1.0/15.8 MB 606.3 kB/s eta 0:00:25
   -- ------------------------------------- 1.0/15.8 MB 606.3 kB/s eta 0:00:25
   --- ------------------------------------ 1.3/15.8 MB 604.4 kB/s eta 0:00:24
   --- ------------------------------------ 1.3/15.8 MB 604.4 kB/s eta 0:00:24
   --- ------------------------------------ 1.3/15.8 MB 604.4 kB/s eta 0:00:24
   --- --

Data akuisisi didapat dari rekaman suara manual
Rekaman dilakukan menggunakan mikrofon.
Setiap perintah suara direkam dalam lingkungan yang minim
noise. File audio disimpan dalam format WAV dan diklasifikasikan
berdasarkan kategori perintah.

In [2]:
import librosa
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import librosa.display

# Fungsi untuk ekstraksi MFCC
def extract_mfcc(y, sr):
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return np.mean(mfcc.T, axis=0)

# Fungsi untuk ekstraksi Spectrogram
def extract_spectrogram(y, sr):
    S = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
    log_S = librosa.power_to_db(S, ref=np.max)
    return log_S

# Fungsi untuk melatih model deteksi suara
def train_sound_model_from_folder(folder_path):
    features = []
    labels = []
    
    # Membaca semua folder di dalam folder utama
    for label in os.listdir(folder_path):
        label_folder = os.path.join(folder_path, label)
        if os.path.isdir(label_folder):
            # Membaca semua file audio di dalam folder
            for audio_file in os.listdir(label_folder):
                audio_path = os.path.join(label_folder, audio_file)
                if audio_file.endswith(".wav"):
                    y, sr = librosa.load(audio_path, sr=None, mono=True)
                    # Ekstraksi fitur MFCC
                    mfcc = extract_mfcc(y, sr)
                    features.append(mfcc)
                    labels.append(label)
    
    # Debugging: Pastikan features berisi data
    print(f'Jumlah fitur yang diekstraksi: {len(features)}')

    if len(features) == 0:
        raise ValueError("Tidak ada data audio yang berhasil diekstraksi.")


    X = np.array(features)
    y = np.array(labels)
    
    # Standardisasi fitur
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    # Pembagian data latih dan data uji
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Model Random Forest
    model = RandomForestClassifier(n_estimators=100)
    model.fit(X_train, y_train)
    
    # Evaluasi model
    accuracy = model.score(X_test, y_test)
    print(f'Akurasi Model: {accuracy * 100:.2f}%')
    
    return model, scaler

# Fungsi untuk deteksi suara berdasarkan model yang telah dilatih
def detect_sound(model, scaler, audio_file):
    y, sr = librosa.load(audio_file, sr=None, mono=True)
    # Ekstraksi MFCC
    mfcc = extract_mfcc(y, sr)
    mfcc_scaled = scaler.transform([mfcc])
    prediction = model.predict(mfcc_scaled)
    
    return prediction[0]

# Contoh penggunaan
folder_path = 'signalwav'  # Ganti dengan path folder audio

# Latih model deteksi suara
model, scaler = train_sound_model_from_folder(folder_path)

# Deteksi suara dalam file audio baru
test_audio = 'matikanlagu5.wav'  # File audio untuk diuji
result = detect_sound(model, scaler, test_audio)

print(f'Suara yang terdeteksi: {result}')


Jumlah fitur yang diekstraksi: 15
Akurasi Model: 66.67%
Suara yang terdeteksi: matikan_lagu
